In [5]:
from collections import defaultdict
from scipy.stats import pearsonr
import numpy as np
import math

#STORE SIMILARITIES IN A DICTIONARY SIM (THIS IS LIKE A SPARSE-MATRIX WHERE WE ONLY STORE NON-ZERO VALUES)
SIM = defaultdict(dict)
#STORE RATINGS in ITM
ITM = defaultdict(dict)

#ITM[m][u] stores rating score for movie m and user u
#SIM[m1][m2] stores similarity score between movie m and m1

ifile = open("netflix-small/ratings-train.txt")
for l in ifile:
    parts = l.strip().split(",")
    ITM[int(parts[0])][int(parts[1])] = float(parts[2])
ifile.close()

for m in ITM.keys():
    for m1 in ITM.keys():
        if m==m1:
            continue
        #COMPUTE SIMILARITY BETWEEN m and m1 and store this value in SIM[m][m1]

ifile = open("netflix-small/ratings-test.txt")
for l in ifile:
    parts = l.strip().split(",")
    movie = parts[0]
    user = parts[1]
    truerating = float(parts[2])
    #FIND K NEIGHBORS for movie from the weights stored in SIM
    #PREDICT the rating by user using the user's ratings for the K neighbors
#Compute the Mean-Squared Error between the true and predicted ratings
ifile.close()